In [1]:
import sys
from kaitoupao_wsl import *

storage dir: /mnt/d/forCoding_data/Tianchi_NLPNewsClassification
code dir: /mnt/d/forCoding_code/Tianchi_NLPNewsClassification 

19 18 27
先天八卦数: 1乾, 2兑, 3离, 4震, 5巽, 6坎, 7艮, 8坤
本卦上：3 本卦下：2 变爻：3


,火泽睽,水火既济,火天大有
上卦,☲离火,☵坎水,☲离火
下卦,☱兑金,☲离火,☰乾金


12 20 10 酉时
先天八卦数: 1乾, 2兑, 3离, 4震, 5巽, 6坎, 7艮, 8坤
本卦上：4 本卦下：4 变爻：4


,雷雷震,水山蹇,地雷复
上卦,☳震木,☵坎水,☷坤土
下卦,☳震木,☶艮土,☳震木


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torch.nn.utils.rnn import pad_sequence
from sklearn.metrics import f1_score

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
device = torch.device("cuda")

# 加载训练集和测试集，将全量字符列表给它弄出来

In [12]:
data_train = pd.read_csv(create_originalData_path("train_set.csv"), sep="\t").sample(10000)
data_test = pd.read_csv(create_originalData_path("test_a.csv"), sep="\t").sample(10000)

In [13]:
data_train.shape, data_test.shape

((10000, 2), (10000, 1))

In [14]:
data_train = data_train.reset_index().rename(columns={"index": "data_id"})
data_test = data_test.reset_index().rename(columns={"index": "data_id"})

## 拆分开

In [15]:
type_of_class = data_train.label.nunique()

In [16]:
train_data, valid_data = train_test_split(data_train, test_size=0.3, random_state=42)

In [17]:
maxlen = 800  # 只考虑每条评论的前 200 个词
def split_list(lst, max_length):
    return [lst[i:i + max_length] for i in range(0, len(lst), max_length)]

In [18]:
from collections import defaultdict
#######################################
new_rst = defaultdict(list)
for idx, row in tqdm.tqdm(train_data.iterrows()):
    idn = row["data_id"]
    lbl = row["label"]
    txt = row["text"].strip().split()
    
    for spt in split_list(txt, maxlen):
        new_rst["data_id"].append(idn)
        new_rst["label"].append(lbl)
        new_rst["text"].append(" ".join(spt))
train_data = pd.DataFrame(new_rst)
#######################################
new_rst = defaultdict(list)
for idx, row in tqdm.tqdm(valid_data.iterrows()):
    idn = row["data_id"]
    lbl = row["label"]
    txt = row["text"].strip().split()
    
    for spt in split_list(txt, maxlen):
        new_rst["data_id"].append(idn)
        new_rst["label"].append(lbl)
        new_rst["text"].append(" ".join(spt))
valid_data = pd.DataFrame(new_rst)
#######################################
new_rst = defaultdict(list)
for idx, row in tqdm.tqdm(data_test.iterrows()):
    idn = row["data_id"]
    # lbl = row["label"]
    txt = row["text"].strip().split()
    
    for spt in split_list(txt, maxlen):
        new_rst["data_id"].append(idn)
        # new_rst["label"].append(lbl)
        new_rst["text"].append(" ".join(spt))
data_test = pd.DataFrame(new_rst)

7000it [00:00, 21631.75it/s]
3000it [00:00, 23744.84it/s]
10000it [00:00, 25398.32it/s]


In [19]:
train_data.shape, valid_data.shape, data_test.shape

((11531, 3), (4968, 3), (16536, 2))

In [20]:
train_labels = torch.tensor(train_data.label.to_list(), dtype=torch.long)
valid_labels = torch.tensor(valid_data.label.to_list(), dtype=torch.long)
test_labels = torch.tensor([-1 for x in range(data_test.shape[0])], dtype=torch.long) ## test_labels的label是假的。

In [21]:
id_train = torch.tensor(list(train_data.data_id), dtype=torch.long)
id_valid = torch.tensor(list(valid_data.data_id), dtype=torch.long)
id_test = torch.tensor(list(data_test.data_id), dtype=torch.long)

## 使用TF-IDF提取特征

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
tfidf_train_features = tfidf_vectorizer.fit_transform(train_data['text'])
tfidf_valid_features = tfidf_vectorizer.transform(valid_data['text'])
tfidf_test_features = tfidf_vectorizer.transform(data_test['text'])

In [23]:
train_features = torch.tensor(tfidf_train_features.toarray(), dtype=torch.float32)
valid_features = torch.tensor(tfidf_valid_features.toarray(), dtype=torch.float32)
test_features = torch.tensor(tfidf_test_features.toarray(), dtype=torch.float32)

In [24]:
sc_input_dim = train_features.shape[1]
sc_input_dim

5111

## 创建适合于语言序列的数据

In [25]:
# 下载数据并进行预处理
vocab_size = 8000  # 只考虑前 20k 词汇

In [26]:
def preprocess_seq_str_2_int(seq, len_lim = maxlen):
    rst = [int(wd) for idx, wd in enumerate(seq.strip().split()) if idx < len_lim]
    return rst

In [27]:
x_train = [torch.tensor(preprocess_seq_str_2_int(seq), dtype=torch.long) for seq in train_data.text]
x_valid = [torch.tensor(preprocess_seq_str_2_int(seq), dtype=torch.long) for seq in valid_data.text]
x_test = [torch.tensor(preprocess_seq_str_2_int(seq), dtype=torch.long) for seq in data_test.text]

In [28]:
x_train = pad_sequence(x_train, batch_first=True, padding_value=0)
x_valid = pad_sequence(x_valid, batch_first=True, padding_value=0)
x_test = pad_sequence(x_test, batch_first=True, padding_value=0)

## 准备数据集

In [29]:
batchsize = 32

In [30]:
class MyData(Dataset):
    def __init__(
        self, 
        ori_data, tfidf_feats, label,
        data_id,
    ):
        self.ori_data = ori_data
        self.tfidf_feats = tfidf_feats
        self.label = label
        self.data_id = data_id
 
    def __len__(self):
        return len(self.ori_data)
 
    def __getitem__(self, idx):
        tuple_ = (
            self.ori_data[idx], 
            self.tfidf_feats[idx], 
            self.label[idx], 
            self.data_id[idx]
        )
        return tuple_

In [31]:
train_loader = DataLoader(MyData(x_train, train_features, train_labels, id_train), batch_size=batchsize, shuffle=True,)
val_loader = DataLoader(MyData(x_valid, valid_features, valid_labels, id_valid), batch_size=batchsize, shuffle=True,)
test_loader = DataLoader(MyData(x_test, test_features, test_labels, id_test), batch_size=batchsize, shuffle=True,)

# 定义网络结构

In [32]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, ff_dim, dropout=0.1):
        super(TransformerBlock, self).__init__()
        self.att = nn.MultiheadAttention(embed_dim, num_heads)
        self.ffn = nn.Sequential(
            nn.Linear(embed_dim, ff_dim),
            nn.ReLU(),
            nn.Linear(ff_dim, embed_dim)
        )
        self.layernorm1 = nn.LayerNorm(embed_dim)
        self.layernorm2 = nn.LayerNorm(embed_dim)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, x):
        attn_output, _ = self.att(x, x, x)
        attn_output = self.dropout1(attn_output)
        out1 = self.layernorm1(x + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        return self.layernorm2(out1 + ffn_output)

In [33]:
class TokenAndPositionEmbedding(nn.Module):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = nn.Embedding(vocab_size, embed_dim)
        self.pos_emb = nn.Embedding(maxlen, embed_dim)

    def forward(self, x):
        maxlen = x.size(1)
        positions = torch.arange(0, maxlen, device=x.device).unsqueeze(0).expand_as(x)
        return self.token_emb(x) + self.pos_emb(positions)

In [34]:
# 单层神经网络：
class SentimentClassifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.fc = nn.Linear(input_dim, output_dim)
 
    def forward(self, x):
        return self.fc(x)

In [35]:
# 定义模型
class TransformerModel(nn.Module):
    def __init__(self, maxlen, vocab_size, embed_dim, num_heads, ff_dim):
        super(TransformerModel, self).__init__()
        self.embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
        self.transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
        self.global_avg_pool = nn.AdaptiveAvgPool1d(1)
        self.dropout1 = nn.Dropout(0.1)
        self.dense1 = nn.Linear(embed_dim, 20)
        self.dropout2 = nn.Dropout(0.1)
        self.dense2 = nn.Linear(20, type_of_class)

        self.sc_net = SentimentClassifier(sc_input_dim, type_of_class)
        
    def forward(self, x, x_tfidf):
        x = self.embedding_layer(x).transpose(0, 1)  # Transformer expects (seq_len, batch_size, embed_dim)
        x = self.transformer_block(x)
        x = x.transpose(0, 1)
        x = self.global_avg_pool(x.permute(0, 2, 1)).squeeze(-1)
        x = self.dropout1(x)
        x = F.relu(self.dense1(x))
        x = self.dropout2(x)
        return F.log_softmax(self.dense2(x) + self.sc_net(x_tfidf), dim=-1)

# 构建模型以及训练

In [36]:
model = TransformerModel(maxlen, vocab_size, embed_dim=64, num_heads=4, ff_dim=32)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [39]:
def get_final_perf(train_list):
    tst_df = pd.DataFrame(train_list)
    
    tst_df1 = tst_df.groupby("dids")["outputs"].apply(lambda x: x.tolist()).reset_index()
    tst_df1["mean_val"] = tst_df1.outputs.apply(lambda x: torch.tensor(x, dtype=torch.float32).mean(axis=0))
    
    tst_df_final = tst_df1.merge(tst_df[["dids", "targets"]].drop_duplicates())
    
    final_outputs = torch.tensor(np.array(tst_df_final.mean_val.to_list()))
    final_targets = torch.tensor(tst_df_final.targets.to_list(), dtype=torch.long)
    
    _, final_predicted = torch.max(final_outputs, 1)
    print(f'accuracy: {final_predicted.eq(final_targets).sum().item()/final_predicted.shape[0]}, f1 score: {f1_score(final_targets, final_predicted, average="macro")}')

In [40]:
# 训练和评估模型
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=2):
    # device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        correct = 0
        total = 0

        train_list = defaultdict(list)
        
        for inputs, input_tfidfs, targets, dids in tqdm.tqdm(train_loader):
            inputs, input_tfidfs, targets = inputs.to(device), input_tfidfs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs, input_tfidfs)
            loss = criterion(outputs, targets)
            loss.requires_grad_(True)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            train_list["outputs"] += outputs.tolist()
            train_list["dids"] += dids.tolist()
            train_list["targets"] += targets.tolist()
    
        # print(f'Epoch {epoch + 1}/{epochs}, Loss: {train_loss/len(train_loader)}, Accuracy: {100.*correct/total}%')
        get_final_perf(train_list)
        
        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            total_predicted = []
            total_label = []
            valid_list = defaultdict(list)
            for inputs, input_tfidfs, targets, dids in tqdm.tqdm(val_loader):
                inputs, input_tfidfs, targets = inputs.to(device), input_tfidfs.to(device), targets.to(device)
                outputs = model(inputs, input_tfidfs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()

                total_predicted += list(predicted)
                total_label += list(targets)

                valid_list["outputs"] += outputs.tolist()
                valid_list["dids"] += dids.tolist()
                valid_list["targets"] += targets.tolist()
        
        # f1 = f1_score(total_label, total_predicted, average='macro')
        # print(f'Validation Loss: {val_loss/len(val_loader)}, Accuracy: {100.*correct/total}%, f1 score is {f1}')
        get_final_perf(valid_list)

In [41]:
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=1)

100%|█████████████████████████████████████████████████████████████████████████████████| 361/361 [00:05<00:00, 70.27it/s]


accuracy: 0.48142857142857143, f1 score: 0.17470232732680085


100%|████████████████████████████████████████████████████████████████████████████████| 156/156 [00:00<00:00, 205.39it/s]


accuracy: 0.5996666666666667, f1 score: 0.2759516619034978


In [42]:
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=5)

100%|█████████████████████████████████████████████████████████████████████████████████| 361/361 [00:04<00:00, 74.74it/s]


accuracy: 0.7091428571428572, f1 score: 0.4065528831331292


100%|████████████████████████████████████████████████████████████████████████████████| 156/156 [00:00<00:00, 188.51it/s]


accuracy: 0.724, f1 score: 0.4480415691144275


100%|█████████████████████████████████████████████████████████████████████████████████| 361/361 [00:04<00:00, 78.11it/s]


...

accuracy: 0.8386666666666667, f1 score: 0.7313501475444829


100%|█████████████████████████████████████████████████████████████████████████████████| 361/361 [00:04<00:00, 76.59it/s]


accuracy: 0.911, f1 score: 0.855352502913665


100%|████████████████████████████████████████████████████████████████████████████████| 156/156 [00:00<00:00, 192.93it/s]

accuracy: 0.853, f1 score: 0.7566818545945541


In [43]:
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=5)

100%|█████████████████████████████████████████████████████████████████████████████████| 361/361 [00:04<00:00, 78.89it/s]


accuracy: 0.9281428571428572, f1 score: 0.8832178584571322


100%|████████████████████████████████████████████████████████████████████████████████| 156/156 [00:00<00:00, 202.04it/s]


accuracy: 0.8456666666666667, f1 score: 0.7774279799736966


100%|█████████████████████████████████████████████████████████████████████████████████| 361/361 [00:04<00:00, 78.07it/s]


...

accuracy: 0.8416666666666667, f1 score: 0.7976382072336917


100%|█████████████████████████████████████████████████████████████████████████████████| 361/361 [00:04<00:00, 78.78it/s]


accuracy: 0.9697142857142858, f1 score: 0.9586089282761393


100%|████████████████████████████████████████████████████████████████████████████████| 156/156 [00:00<00:00, 207.30it/s]

accuracy: 0.8326666666666667, f1 score: 0.7849451738780281


In [ ]:
# train_model(model, train_loader, val_loader, criterion, optimizer, epochs=1)

In [ ]:
# train_model(model, train_loader, val_loader, criterion, optimizer, epochs=1)

# 保存模型

In [ ]:
save_pickle_object(model, create_trained_models_path("ori_9041-dev_tfidf_lenlim.pkl"))

In [ ]:
run_finish()

# 预测一下试试

In [ ]:
## 这里还要改一下的哦。

In [ ]:
with torch.no_grad():
    total_predicted = []
    for inputs, input_tfidfs, targets in tqdm.tqdm(test_loader):
        outputs = model(inputs, input_tfidfs)
        _, predicted = torch.max(outputs, 1)
        total_predicted += list(predicted)

In [ ]:
oot_rst = [int(x) for x in total_predicted]
len(oot_rst)

In [ ]:
store_data_to_newbasepath(pd.DataFrame({"label": oot_rst}), "rst-20250119_1", fmt="csv")